In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd
from os.path import join, basename

sys.path.append(str(Path(getcwd()).parent.parent))

In [ ]:
import numpy as np
import pandas as pd
import re
from unidecode import unidecode
from datetime import date, timedelta, datetime
from os.path import dirname, join, isfile, isdir
from bulletin import default_input, default_output

path_file = join('C:\\', 'SESA', 'covid19datascience-dev', 'bulletin', 'database', 'gal')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

In [ ]:
# COLUNAS DO BANCO DO GAL

# Index(['id'0, 'origem_gal'1, 'requisicao'2, 'data_requisicao'3, 'data_solicitacao'4,
#         'requisicao_correlativo'5, 'unidade_requisitante'6,
#         'unidade_requisitante_municipio'7, 'unidade_requisitante_uf'8, 'paciente'9,
#         'cns'10, 'cpf'11, 'nome_mae'12, 'sexo'13, 'data_nascimento'14, 'idade'15,
#         'raca_cor'16, 'idade_gestacional'17, 'ibge'18, 'municipio'19, 'latitude'20,
#         'longitude'21, 'codigo_regional'22, 'regional'23, 'data_inicio_sintomas'24,
#         'cod_exame_requisicao'25, 'data_coleta'26, 'codigo_amostra'27,
#         'numero_amostra'28, 'numero_interno'29, 'material_biologico'30,
#         'data_encaminhamento_amostra'31, 'data_recebimento_amostra'32,
#         'tempo_transporte_amostra'33, 'envio_oportuno'34, 'exame'35, 'metodo'36,
#         'resultado'37, 'virus'38, 'data_liberacao'39, 'laboratorio_executor'40,
#         'laboratorio_executor_municipio'41, 'correlativo_exame'42,
#         'tempo_liberacao'43, 'tempo_diagnostico'44, 'codigo_status'45, 'status'46,
#         'classificacao'47],
#        dtype='object')]

In [ ]:
gal_2020 = pd.read_pickle(join(path_file, 'banco_gal_2020-bz2.pkl'), compression='bz2')
gal_2020 = gal_2020[['id', 'data_requisicao', 'paciente', 'cns', 'cpf', 'nome_mae', 'sexo', 'idade', 'municipio', 'regional', 'exame', 'metodo', 'resultado', 'virus', 'data_liberacao', 'laboratorio_executor', 'status', 'classificacao']]

In [ ]:
gal_2021 = pd.read_pickle(join(path_file, 'banco_gal_2021-bz2.pkl'), compression='bz2')
gal_2021 = gal_2020[['id', 'data_requisicao', 'paciente', 'cns', 'cpf', 'nome_mae', 'sexo', 'idade', 'municipio', 'regional', 'exame', 'metodo', 'resultado', 'virus', 'data_liberacao', 'laboratorio_executor', 'status', 'classificacao']]

In [ ]:
gal_complemento = pd.read_csv(join(path_file, 'consulta-gal-complemento.csv'), sep = ';', usecols=['id', 'data_requisicao', 'paciente', 'cns', 'cpf', 'nome_mae', 'sexo', 'idade', 'municipio', 'regional', 'exame', 'metodo', 'resultado', 'virus', 'data_liberacao', 'laboratorio_executor', 'status', 'classificacao'])

In [ ]:
gal = pd.concat([gal_2020, gal_2021, gal_2021_complemento])
gal[['id', 'data_requisicao']].groupby('data_requisicao').count().sort_values('data_requisicao')

In [ ]:
boletim = gal.loc[gal['virus'] == 'Coronavírus SARS-CoV2']
boletim = boletim.loc[boletim['resultado'] == 'Detectável']
# boletim = boletim.loc[boletim['unidade_requisitante_uf'] == 'PARANA']

In [ ]:
# CPF não nulo
cpf_nao_nulo = boletim.loc[boletim['cpf'].notna()]

# CPF nulo, mas que contenha CNS
cpf_nulo = boletim.loc[boletim['cpf'].isna()]
cpf_nulo = cpf_nulo.loc[cpf_nulo['cns'].notna()]

In [ ]:
duplicados_cpf = cpf_nao_nulo.copy()
duplicados_cns = cpf_nulo.copy()

In [ ]:
duplicados_cpf = duplicados_cpf.loc[duplicados_cpf.duplicated(subset=['cpf'], keep=False)]
duplicados_cns = duplicados_cns.loc[duplicados_cns.duplicated(subset=['cns'], keep=False)]

In [ ]:
searched_date = 'data_requisicao'

In [ ]:
duplicados_cpf[searched_date] = pd.to_datetime(duplicados_cpf[searched_date], format='%Y-%m-%d', errors='coerce')
duplicados_cns[searched_date] = pd.to_datetime(duplicados_cns[searched_date], format='%Y-%m-%d', errors='coerce')

duplicados_cpf = duplicados_cpf.sort_values(searched_date, ascending=True)
duplicados_cns = duplicados_cns.sort_values(searched_date, ascending=True)

In [ ]:
exame_temp = pd.DataFrame()
temporario = pd.DataFrame()
reinfeccao_cpf = pd.DataFrame()
reinfeccao_cns = pd.DataFrame()

cpf_unico = duplicados_cpf['cpf'].unique()
cns_unico = duplicados_cns['cns'].unique()
print('CPFs únicos encontrados = ', len(cpf_unico))
print('CNSs únicos encontrados = ', len(cns_unico))

In [ ]:
# for i,paciente in enumerate(cpf_unico):
for paciente in cpf_unico:
    # print(i,end=' ')
    # exame_temp = pd.DataFrame()
    exame_temp = duplicados_cpf.loc[duplicados_cpf['cpf'] == paciente].sort_values(searched_date, ascending=True)
    exame_1 = exame_temp.iloc[0]
    exame_2 = exame_temp.iloc[-1]
    dif = exame_2[searched_date] - exame_1[searched_date]
    # print(dif.days)
    if (dif.days > 90):
        temporario = duplicados_cpf.loc[(duplicados_cpf['cpf'] == paciente) & (duplicados_cpf[searched_date] == exame_2[searched_date])]
        reinfeccao_cpf = reinfeccao_cpf.append(temporario)

In [ ]:
for paciente in cns_unico:
    # print(i,end=' ')
    exame_temp = pd.DataFrame()
    exame_temp = duplicados_cns.loc[duplicados_cns['cns'] == paciente].sort_values(searched_date, ascending=True)
    exame_1 = exame_temp.iloc[0]
    exame_2 = exame_temp.iloc[-1]
    dif = exame_2[searched_date] - exame_1[searched_date]
    # print(dif.days)
    if (dif.days > 90):
        temporario = duplicados_cns.loc[(duplicados_cns['cns'] == paciente) & (duplicados_cns[searched_date] == exame_2[searched_date])]
        reinfeccao_cns = reinfeccao_cns.append(temporario)

In [ ]:
reinfeccao_cpf.drop_duplicates(subset=['cpf'], keep='last')
reinfeccao_cns.drop_duplicates(subset=['cns'], keep='last')

reinfeccao = pd.concat([reinfeccao_cpf, reinfeccao_cns])
reinfeccao = reinfeccao.sort_values(searched_date, ascending=True)
reinfeccao

In [ ]:
print('----- CASOS DE *POSSÍVEIS* REINFECÇÕES NO PARANÁ -----')
print('*TOTAL* =',len(reinfeccao))
print('*Fonte:* Banco de dados do GAL, dia 30/06/2021.')
print('*Metodologia:*')
print('Foram obtidos CPF’s e CNS’s únicos dentro do Sistema do GAL. Desse montante, foi verificado exames com resultado “Detectável” para o vírus “Coronavírus SARS-CoV2” para uma mesma pessoa. Aqueles exames que tiveram uma *data de requisição* superior ou igual a 90 dias do primeiro resultado “Detectável” para o vírus, foi considerado como um caso de reincidência de confirmação para o Coronavírus SARS-CoV2.')

In [ ]:
# JOÃO

# cpf_unico = duplicados_cpf['cpf'].unique()
# for paciente in cpf_unico:
#     # exame_temp = pd.DataFrame()
#     exame_temp = duplicados_cpf.loc[duplicados_cpf['cpf'] == paciente].sort_values('data_requisicao')  
#     count = 0
#     for dt in exame_temp['data_requisicao']:
#         data_temp = dt
#         if (count == 0):
#             data1 = dt
#         dif = data_temp - data1
#         count = count + 1
#         print(dif.days)

# temp = duplicados_cpf.groupby(['cpf'])[['paciente']].count().reset_index()
# temp['cpf'] = temp['cpf'].astype('str')
# temp.loc[temp['paciente']>=9]